In [376]:
import re 
import numpy as np
from collections import defaultdict
from collections import Counter
%matplotlib inline
from IPython.core.debugger import set_trace

In [209]:
class Tree():
    """Implementation of a simple tree. 
       i.e. the stack is inverted"""
    def __init__(self ): 
        self.nodes = {}
        self.links = {}
    
    def add_node(self, name, weight):
        self.nodes[name] = weight
        
    # we want to go from smallest to largest node, or top to bottom.
    # so link a node to the node holding it up 
    def add_link(self, name_above, name_below):
        self.links[name_above] = name_below
        

In [314]:
def create_tree(txt):
    t = Tree()
    
    disc_name_string = '^(\w+)'
    disc_weight_string ='\((\d+)\)'
    
    for line in txt: 
        disc_name = re.findall(disc_name_string, line)[0]
        disc_weight = int(re.findall(disc_weight_string, line)[0])
        # Put the node in tree
        t.add_node(disc_name,  disc_weight)

    # Add links on second pass 
    for line in txt: 
        disc_below = re.findall(disc_name_string, line)[0]
        if line.find('->') != -1 :
            _, disc_string_above = line.split(' -> ')
            disc_name_above_list = disc_string_above.split(', ')

            for disc_above in disc_name_above_list:
                t.add_link(disc_above, disc_below)
                
    return t

In [317]:
def find_bottom_program(t): 
    # If a node has no parent, it is the bottom program. 
    # We can start anywhere and go until we hit a program without a parent 
    current_node = list(t.nodes.keys())[0]
    
    flag = True
    while flag:
        try:
            current_node = t.links[current_node]
        except KeyError:
            flag = False 
            return current_node
    

In [322]:
# Test input 
# One takeaway is to really try and structure programs so that they work on 
# different inputs easily. 
txt = [x.strip('\n') for x in open('input_2017_07_test.txt').readlines()]
t = create_tree(txt)
bottom_program = find_bottom_program(t)
assert bottom_program == 'tknk'

In [323]:
txt = [x.strip('\n') for x in open('input_2017_07.txt').readlines()]
t = create_tree(txt)
bottom_program = find_bottom_program(t)
print(bottom_program)

svugo


## Part 2 

This is harder...i have my links going from high to low, stored in a dict. This task I need to search from low to high, so I need the links the other way around. 

First thought is to just reverse the links...but this can't really be done in dict form, since we will have duplicate keys. We can have a dict linking to arrays? Or have a two dimensional list, or a numpy version. 

I like the dict version linking to arrays. Let's do that. 

In [326]:
def reverse_links(t):
    # Set up dict, with a list with every empty value 
    links_below_above = defaultdict(list)

    for disc_above, disc_below in t.links.items():
        links_below_above[disc_below].append(disc_above)
    return links_below_above

Now we can go up the tree and calculate the weight at each bit

In [295]:
node_cumulative_weight = defaultdict(lambda: 0)

In [371]:
def find_weight(t, node_name): 
    global indent_level
    #set_trace()
    # case where we have a program on the top   
    # if you're at the top, just return your weight (you have nothing above you)
    #print ("".join([" " for i in range(indent_level)]) + node_name)
    if links_below_above[node_name] == []:  
        node_cumulative_weight[node_name] = t.nodes[node_name]
        return t.nodes[node_name]
    # if the node has ones above, we need to get weight of the nodes 
    else:
        node_sum = t.nodes[node_name]
        for node_above in links_below_above[node_name]:
            # the case where we already computed the weight 
            if node_cumulative_weight[node_above] != 0: 
                node_sum +=  node_cumulative_weight[node_above]
            # we haven't computed the weight; do so 
            else: 
                indent_level +=1 
                node_sum += find_weight(t, node_above)
        node_cumulative_weight[node_name] = node_sum
        indent_level -= 1
        return node_sum

In [427]:
def find_unbalanced_programs(t, node_name):
    # start at root. 
    # keep going down while unbalanced 
    # if nodes above are balanced, it's the node itself that is the problem
    # then look at the nodes below that node to find what it should be 
    global indent_level
    nodes_above_weight = [node_cumulative_weight[node_name] 
                          for node_name in links_below_above[node_name]]
    
    # find the different one 
    c = Counter(nodes_above_weight)
    most_common_value = c.most_common(1)[0][0]
    
    # case where all values are the same 
    # this is the broken node
    if c.most_common(1)[0][1] == len(nodes_above_weight): 
        print (node_name)
        # we want to find the level below to see what its weight should be 
        below_node = t.links[node_name]
        # find weights of the parallel 
        nodes_above_weight_below = [node_cumulative_weight[node_name] 
                          for node_name in links_below_above[below_node]]
        c = Counter(nodes_above_weight_below)
        most_common_value_below = c.most_common(1)[0][0]
        different_index = ([i for i, x in enumerate(nodes_above_weight_below) 
                        if x!=most_common_value_below ])[0] 
        
        # Case; stack is too light 
        node_diff = (max(nodes_above_weight_below) - min(nodes_above_weight_below))
        #set_trace()
        if max(nodes_above_weight_below) == most_common_value: 
            return (t.nodes[node_name] + node_diff)
        else:       # Case; stack is too heavy 
            return (t.nodes[node_name] - node_diff)
        
    # case where one value is different 
    # we need to go down one level 
    # get the index of the different value 
    different_index = ([i for i, x in enumerate(nodes_above_weight) 
                        if x!=most_common_value ])[0] 
    
    name_above = links_below_above[node_name][different_index]
    return find_unbalanced_programs(t, name_above)

Let's try on the test input.

In [428]:
txt = [x.strip('\n') for x in open('input_2017_07_test.txt').readlines()]
t = create_tree(txt)
links_below_above = reverse_links(t)
node_cumulative_weight = defaultdict(lambda: 0)
indent_level = 0 
# Use root, weights are stored in node_cumulative_weight
find_weight(t, 'tknk')
find_unbalanced_programs(t, 'tknk')

ugml


60

Now the real one

In [429]:
txt = [x.strip('\n') for x in open('input_2017_07.txt').readlines()]
t = create_tree(txt)
links_below_above = reverse_links(t)
node_cumulative_weight = defaultdict(lambda: 0)
indent_level = 0 
# Use root, weights are stored in node_cumulative_weight
find_weight(t, 'svugo')
find_unbalanced_programs(t, 'svugo')

sphbbz


1152

### Digraph approach (obsolete)

In [213]:
# # Digraph approach (replaced by custom tree)
# G = nx.DiGraph()
# # Add nodes on first pass, links on second pass 
# for line in txt: 
#     disc_name = re.findall(disc_name_string, line)[0]
#     disc_weight = int(re.findall(disc_weight_string, line)[0])
#     # Put the node in graph
#     G.add_node(disc_name, {'weight': disc_weight})

    
# # Add links on second pass 
# for line in txt: 
#     disc_name = re.findall(disc_name_string, line)[0]
#     if line.find('->') != -1 :
#         _, disc_string_above = line.split(' -> ')
#         disc_name_above_list = disc_string_above.split(', ')
        
#         for disc_above in disc_name_above_list:
#             G.add_edge(disc_name, disc_above)